In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from test_utils import test
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets

# Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1))
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [3]:
def hinge_loss(y, X, w):
    return np.clip(1 - y * (X @ w), 0, np.inf)


def calculate_primal_objective(y, X, w, lambda_):
    """Compute the full cost (the primal objective,
    equation (1) in the exercise pdf), that is loss plus regularizer.

    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)

    Returns:
        scalar, non-negative

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_primal_objective(y_test, x_test, w_test, 1)
    49.0
    """
    return np.sum(hinge_loss(y, X, w)) + lambda_ * np.linalg.norm(w)**2 / 2

In [4]:
test(calculate_primal_objective)

✅ Your `calculate_primal_objective` passed 4 tests.


In [5]:
def calculate_accuracy(y, X, w):
    """Compute the accuracy on the given dataset (X, y) using the model w.

    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)

    Returns:
        scalar, between 0 and 1

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_accuracy(y_test, x_test, w_test)
    0.5
    """
    y_pred = (X @ w > 0) * 2 - 1
    return np.mean(y_pred == y)

In [6]:
test(calculate_accuracy)

✅ Your `calculate_accuracy` passed 4 tests.


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

Subgradient:
$$
N\cdot \nabla \ell\left(y_nx_n^Tw\right) + \lambda w
$$

Where:
$$
\nabla \ell\left(y_nx_n^Tw\right) =
\begin{cases}
-y_nx_n &\quad 1-y_nx_n^Tw > 0\\
0 &\quad \text{otherwise}
\end{cases}
$$

In [7]:
def hinge_loss_subgradient(y, X, w):
    if y * (X @ w) < 1:
        return -y * X.T
    return np.zeros(w.size)


def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """Compute the stochastic gradient of loss plus regularizer.

    Args:
        X: the dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
        lambda_: positive scalar number
        n: the index of the (one) datapoint we have sampled
        num_examples: N

    Returns:
        numpy array, shape = (num_features)

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_stochastic_gradient(y_test, x_test, w_test, 1, 1, 2)
    array([ 8, 11, 17])
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum,
    # not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    return num_examples * hinge_loss_subgradient(y_n, x_n, w) \
        + lambda_ * w

In [8]:
test(calculate_stochastic_gradient)

✅ Your `calculate_stochastic_gradient` passed 4 tests.


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [9]:
def sgd_for_svm_demo(y, X):
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)
    # big because scales with N due to the formulation of the problem
    # (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)

    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0, num_examples - 1)

        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it + 1) * grad

        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))

    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))


sgd_for_svm_demo(y, X)

iteration=0, cost=44625835.47062729
iteration=20000, cost=370.53342721124125
iteration=40000, cost=299.2527293477587
iteration=60000, cost=137.61507525078594
iteration=80000, cost=130.88365808879507
iteration=100000, cost=132.1276222125438
iteration=120000, cost=138.69328157805387
iteration=140000, cost=125.18803444398534
iteration=160000, cost=124.83972469443253
iteration=180000, cost=147.9647256928693
training accuracy = 0.9191564147627417


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [ ]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """Compute a coordinate update (closed form) for coordinate n.

    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        lambda_: positive scalar number
        alpha: vector of dual coordinates, shape = (num_examples)
        w: vector of primal parameters, shape = (num_features)
        n: the coordinate to be updated

    Returns:
        w: updated vector of primal parameters, shape = (num_features)
        alpha: updated vector of dual parameters, shape = (num_examples)

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([0., 1., 5.])
    >>> alpha_test = np.array([5., 3.])
    >>> calculate_coordinate_update(y_test, x_test, 1, alpha_test, w_test, 0)
    (array([-4., -7., -7.]), array([1., 3.]))
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])

    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    raise NotImplementedError

    return w, alpha

In [ ]:
test(calculate_coordinate_update)

In [11]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """Calculate the objective for the dual problem.

    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        w: vector of primal parameters, shape = (num_features)
        alpha: vector of dual coordinates, shape = (num_examples)
        lambda_: non negative scalar number

    Output:
        scalar

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([0., 1., 5.])
    >>> alpha_test = np.array([5., 3.])
    >>> calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
    -5.0
    """
    return np.sum(alpha) - alpha.T @ y * X @ X.T @ y @ alpha / (2 * lambda_)

In [12]:
test(calculate_dual_objective)

❌ The are some issues with your implementation of `calculate_dual_objective`:
**********************************************************************
File "__main__", line 18, in calculate_dual_objective
Failed example:
    calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
Expected:
    -5.0
Got:
    233.0
**********************************************************************


In [ ]:
# Notice that the gap is going to 0
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 20000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)